## **Setting up Development Environment**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd /content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1

/content/drive/MyDrive/Colab Notebooks/Hate-Speech-Detection-in-Tamil/HASOC Tamil/Task 1


In [3]:
! rm -rf indic_nlp_resources
! git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 14.68 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Checking out files: 100% (28/28), done.


In [4]:
! pip install -U -q transformers
! pip install -U -q sentencepiece
! pip install -U -q indic-nlp-library

     |████████████████████████████████| 4.4 MB 30.8 MB/s 
     |████████████████████████████████| 6.6 MB 43.5 MB/s 
     |████████████████████████████████| 101 kB 10.1 MB/s 
     |████████████████████████████████| 596 kB 55.3 MB/s 
     |████████████████████████████████| 1.2 MB 37.0 MB/s 
     |████████████████████████████████| 40 kB 5.3 MB/s 
     |████████████████████████████████| 2.8 MB 29.6 MB/s 


## **Importing Libraries**

In [71]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from indicnlp import common
from indicnlp import loader
from indicnlp.morph import unsupervised_morph

from transformers import AutoTokenizer, AutoModel

import pickle

from sklearn.model_selection import GridSearchCV

## **Setting up Stemmer**

In [6]:
INDIC_NLP_RESOURCES = "./indic_nlp_resources"
common.set_resources_path(INDIC_NLP_RESOURCES)
loader.load()
analyzer = unsupervised_morph.UnsupervisedMorphAnalyzer('ta')

## **Loading Data**

In [7]:
dataframe = pd.read_csv("Dataset/tamil_offensive_train.tsv", sep='\t')

In [8]:
dataframe.head()

,id,text,category
0,tam1,திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...,NOT
1,tam2,இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...,OFF
2,tam3,மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...,NOT
3,tam4,மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......,OFF
4,tam5,only for விஜய் சேதுபதி and STR,NOT


In [9]:
dataframe.describe()

,id,text,category
count,5880,5880,5880
unique,5880,5812,3
top,tam1,வந்தா ராஜாவா தான் வருவேன்........ ராஜாவா வந்தா...,NOT
freq,1,5,4724


In [10]:
dataframe['category'].unique()

array(['NOT', 'OFF', 'not-Tamil'], dtype=object)

In [11]:
dataframe.shape

(5880, 3)

In [12]:
dataframe = dataframe.loc[dataframe['category'] != 'not-Tamil']
dataframe['category'].unique()

array(['NOT', 'OFF'], dtype=object)

In [13]:
dataframe.shape

(5877, 3)

In [14]:
text = dataframe['text']

In [15]:
text

0       திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற...
1       இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இத...
2       மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வ...
3       மொத்த சாதியும் ஒரு சாதிக்கு எதிரா நிக்குது.......
4                          only for விஜய் சேதுபதி and STR
                              ...                        
5875    இப்போதான் ஜாதி பிரச்சினை இல்லாம இருக்கு இப்போ ...
5876    இது ஒரு சாதி ரீதியான படம் இல்லை தான் செல்லும் ...
5877    கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை ...
5878    அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.....
5879    உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங...
Name: text, Length: 5877, dtype: object

In [16]:
label = dataframe['category']

In [17]:
label

0       NOT
1       OFF
2       NOT
3       OFF
4       NOT
       ... 
5875    NOT
5876    NOT
5877    NOT
5878    OFF
5879    NOT
Name: category, Length: 5877, dtype: object

## **Data Preprocessing**

In [18]:
le = LabelEncoder()
label = le.fit_transform(label)
label

array([0, 1, 0, ..., 0, 1, 0])

In [19]:
text = text.str.replace(r"[+/#@&*$%:]",'')
text = text.to_numpy()
text

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


array(['திருமலை நாயக்கர் பேரவை சார்பாக படம் வெற்றி பெற வாழ்த்துக்கள்',
       'இந்த ட்ரெய்லர் கூட பார்க்கிற மாதிரி இல்லை.. இதை தியேட்டர் ல போய் பார்க்கனுமா.. .',
       'மைசூரு செட்டியார் சமூகத்தின் சார்பாக இப்படம் வெற்றிபெற வாழ்த்துகிறேன்',
       ...,
       'கதை தெரியவில்லை  இருந்தாலும் மாறுபட்ட சிந்தனை வாழ்த்துக்கள்  பறையர் சார்பாக  நன்றி மோகன் ஜி',
       'அட பரவால்லையே 8.5 டிஸ்லைக்ஸ் போட்ருக்கானுங்க.. பரவால வயிறு எறிஞ்சி சாவுங்கடா சாவுங்கடா சாக்காடை நாய்ங்களா..இன்னும் நிறைய டிஸ்லைக்ஸ் எதிர் பாக்குறன்..',
       'உதயநிதிக்கு நடிப்பு வரலனு சொல்ரவங்க லைக் போடுங்கள் ...'],
      dtype=object)

In [20]:
text = text.tolist()
stemmed_text = []

for line in text:
    line = line.split()
    stemmed_line = ''
    for token in line:
        token = analyzer.morph_analyze_document(token.split(' '))[0]
        stemmed_line += token + ' '
    stemmed_text.append(stemmed_line)

text = np.array(stemmed_text)

In [21]:
# text = np.array(text)
# text

## **Feature Extraction**

In [22]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = AutoModel.from_pretrained("google/muril-base-cased")

Downloading:   0%|          | 0.00/206 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/909M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
# Sample Output

tokenized_input = tokenizer(
        text[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [24]:
print(tokenized_input)

{'input_ids': tensor([[  104, 78708, 40896, 56772, 26847,  7772,  6396, 14801, 94111,  1240,
           105]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [25]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-1.7481e-03,  2.0140e-03,  6.5913e-06,  ..., -1.1483e-02,
          -2.0360e-03, -2.8953e-03],
         [ 4.4569e-03, -6.9588e-04,  1.5881e-03,  ..., -1.0344e-02,
           3.2410e-03, -1.2408e-03],
         [ 7.9225e-03,  3.7540e-03,  1.5639e-03,  ..., -1.9098e-02,
          -1.7169e-03,  1.9726e-03],
         ...,
         [-1.8930e-02,  1.2478e-03, -6.0981e-03,  ..., -1.5328e-02,
          -2.8436e-03, -7.3703e-03],
         [ 7.2786e-04,  8.8155e-04, -6.0271e-04,  ..., -2.0038e-02,
          -1.9423e-03, -5.0617e-03],
         [-1.7481e-03,  2.0140e-03,  6.5898e-06,  ..., -1.1483e-02,
          -2.0360e-03, -2.8953e-03]]], grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 9.4623e-03,  1.9147e-02, -7.8154e-03,  1.5501e-02, -6.7766e-03,
         -8.3276e-03, -2.1607e-03, -4.3346e-03, -3.1784e-03, -9.1843e-03,
         -9.6819e-03,  5.2610e-03,  1.5258e-03,  4.3230e-06, -4.8025e-03,
          1.6861e-

In [26]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [27]:
# Tokenizing Input Data

input = []

for i in text:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [28]:
# Embedding the Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./TrainData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./TrainData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./TrainData/output1.pickle done
./TrainData/output2.pickle done
./TrainData/output3.pickle done
./TrainData/output4.pickle done
./TrainData/output5.pickle done
./TrainData/output6.pickle done
./TrainData/output7.pickle done
./TrainData/output8.pickle done
./TrainData/output9.pickle done
./TrainData/output10.pickle done
./TrainData/output11.pickle done
./TrainData/output12.pickle done
./TrainData/output13.pickle done
./TrainData/output14.pickle done
./TrainData/output15.pickle done
./TrainData/output16.pickle done
./TrainData/output17.pickle done
./TrainData/output18.pickle done
./TrainData/output19.pickle done
./TrainData/output20.pickle done
./TrainData/output21.pickle done
./TrainData/output22.pickle done
./TrainData/output23.pickle done
./TrainData/output24.pickle done
./TrainData/output25.pickle done
./TrainData/output26.pickle done
./TrainData/output27.pickle done
./TrainData/output28.pickle done
./TrainData/output29.pickle done
./TrainData/output30.pickle done


In [29]:
# Loading Embedded Input Data from Disk

output = []

for i in range(30):
    file_name = "./TrainData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X = output
output = []

./TrainData/output1.pickle done
./TrainData/output2.pickle done
./TrainData/output3.pickle done
./TrainData/output4.pickle done
./TrainData/output5.pickle done
./TrainData/output6.pickle done
./TrainData/output7.pickle done
./TrainData/output8.pickle done
./TrainData/output9.pickle done
./TrainData/output10.pickle done
./TrainData/output11.pickle done
./TrainData/output12.pickle done
./TrainData/output13.pickle done
./TrainData/output14.pickle done
./TrainData/output15.pickle done
./TrainData/output16.pickle done
./TrainData/output17.pickle done
./TrainData/output18.pickle done
./TrainData/output19.pickle done
./TrainData/output20.pickle done
./TrainData/output21.pickle done
./TrainData/output22.pickle done
./TrainData/output23.pickle done
./TrainData/output24.pickle done
./TrainData/output25.pickle done
./TrainData/output26.pickle done
./TrainData/output27.pickle done
./TrainData/output28.pickle done
./TrainData/output29.pickle done
./TrainData/output30.pickle done


In [30]:
X = np.array(X)
X.shape

(5877, 1, 768)

In [31]:
X = X.reshape(5877, 768)
X.shape

(5877, 768)

In [32]:
y = label
y.shape

(5877,)

## **Train Test Split**

In [33]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
X_train.shape

(4701, 768)

In [35]:
X_val.shape

(1176, 768)

## **Logistic Regression**

In [82]:
lr = LogisticRegression(C=10, max_iter=1000, penalty='none', solver='sag')
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_val)
print(classification_report(y_val, lr_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1484: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  "Setting penalty='none' will ignore the C and l1_ratio parameters"


              precision    recall  f1-score   support

           0       0.86      0.95      0.90       950
           1       0.62      0.33      0.43       226

    accuracy                           0.83      1176
   macro avg       0.74      0.64      0.67      1176
weighted avg       0.81      0.83      0.81      1176



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


## **Support Vector Machine**

In [84]:
svm = SVC(C=0.1, decision_function_shape='ovo', max_iter=1000)
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_val)
print(classification_report(y_val, svm_pred))

              precision    recall  f1-score   support

           0       0.81      1.00      0.89       950
           1       0.00      0.00      0.00       226

    accuracy                           0.81      1176
   macro avg       0.40      0.50      0.45      1176
weighted avg       0.65      0.81      0.72      1176



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Naive Bayes**

In [38]:
nb = GaussianNB()
nb.fit(X_train, y_train)
nb_pred = nb.predict(X_val)
print(classification_report(y_val, nb_pred))

              precision    recall  f1-score   support

           0       0.94      0.73      0.82       950
           1       0.41      0.81      0.54       226

    accuracy                           0.74      1176
   macro avg       0.68      0.77      0.68      1176
weighted avg       0.84      0.74      0.77      1176



## **Stochastic Gradient Descent**

In [89]:
sgd = SGDClassifier(alpha=1e-05, loss='perceptron', penalty='elasticnet')
sgd.fit(X_train, y_train)
sgd_pred = sgd.predict(X_val)
print(classification_report(y_val, sgd_pred))

              precision    recall  f1-score   support

           0       0.85      0.95      0.90       950
           1       0.57      0.28      0.38       226

    accuracy                           0.82      1176
   macro avg       0.71      0.62      0.64      1176
weighted avg       0.79      0.82      0.80      1176



## **K Nearest Neighbours**

In [91]:
knn = KNeighborsClassifier(p=1, weights='distance')
knn.fit(X_train, y_train)
knn_pred = knn.predict(X_val)
print(classification_report(y_val, knn_pred))

              precision    recall  f1-score   support

           0       0.83      0.96      0.89       950
           1       0.54      0.19      0.29       226

    accuracy                           0.81      1176
   macro avg       0.69      0.58      0.59      1176
weighted avg       0.78      0.81      0.78      1176



## **Decision tree**

In [93]:
dt = DecisionTreeClassifier(criterion='entropy', max_features='auto')
dt.fit(X_train, y_train)
dt_pred = dt.predict(X_val)
print(classification_report(y_val, dt_pred))

              precision    recall  f1-score   support

           0       0.84      0.84      0.84       950
           1       0.33      0.35      0.34       226

    accuracy                           0.74      1176
   macro avg       0.59      0.59      0.59      1176
weighted avg       0.74      0.74      0.74      1176



## **Random Forest**

In [96]:
# parameters = {
#     'n_estimators': (100, 150, 200),
#     'criterion': ('gini', 'entropy', 'log_loss'),
#     'max_features': ('sqrt', 'log2')
# }
# 
# rf = RandomForestClassifier()
# clf = GridSearchCV(rf, parameters, n_jobs=4)
# clf.fit(X_train, y_train)
# 
# print(clf.best_estimator_)

In [95]:
rf = RandomForestClassifier(criterion='entropy', max_features='sqrt', n_estimators=200)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_val)
print(classification_report(y_val, rf_pred))

              precision    recall  f1-score   support

           0       0.83      0.99      0.90       950
           1       0.73      0.15      0.24       226

    accuracy                           0.83      1176
   macro avg       0.78      0.57      0.57      1176
weighted avg       0.81      0.83      0.78      1176



## **Majority Voting**

In [99]:
mv_pred = []

for i in range(len(lr_pred)):
    one = 0
    zero = 0
    # Keeping only RF, KNN & NB in predictions due to poor performance
    predictions = [lr_pred[i], sgd_pred[i], dt_pred[i], rf_pred[i], nb_pred[i]]
    # predictions = [lr_pred[i], sgd_pred[i], rf_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_pred.append(1)
    else: mv_pred.append(0)

mv_pred = np.array(mv_pred)

print(classification_report(y_val, mv_pred))    

              precision    recall  f1-score   support

           0       0.85      0.95      0.90       950
           1       0.60      0.30      0.40       226

    accuracy                           0.83      1176
   macro avg       0.72      0.62      0.65      1176
weighted avg       0.80      0.83      0.80      1176



## **Loading Testing Data**

In [100]:
dataframe_test = pd.read_csv(
    "Dataset/tam_offesive_withoutlabels_test.tsv", 
    sep='\t'
)
dataframe_test.head()

,id,text
0,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
1,tam_test_2,நாயுடு மக்கள் சார்பாக திரைப்படம் வெற்றி பெற வா...
2,tam_test_3,தில்லானா முயற்சி தஞ்சை கோனார்
3,tam_test_4,திரௌபதியின் துகிலுறித்த போது காத்த கண்ணனின் யா...
4,tam_test_5,நான் தியேட்டர்லே படம் பார்த்து 35 வருஷமாச்சு! ...


In [101]:
dataframe_test.describe()

,id,text
count,654,654
unique,654,654
top,tam_test_1,தலைவா STR இதுக்குதான் கதுருந்தோம் மாஸ் தலைவா t...
freq,1,1


In [102]:
text_test = dataframe_test['text']

## **Test Data Preprocessing**

In [103]:
text_test = text_test.str.replace(r"[+/#@&*$%:]",'')
text_test = text_test.to_numpy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [104]:
text_test = text_test.tolist()
stemmed_text = []

for line in text_test:
    line = line.split()
    stemmed_line = ''
    for token in line:
        token = analyzer.morph_analyze_document(token.split(' '))[0]
        stemmed_line += token + ' '
    stemmed_text.append(stemmed_line)

text_test = np.array(stemmed_text)

## **Test Feature Extraction**

In [105]:
# Load Transformer Model

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = AutoModel.from_pretrained("google/muril-base-cased")

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [106]:
# Sample Output

tokenized_input = tokenizer(
        text_test[0],
        padding=True,
        truncation=False,
        return_tensors='pt'
    )

sample_output = model(**tokenized_input)

In [107]:
print(tokenized_input)

{'input_ids': tensor([[   104,  20005,  39159,   4095,   1394,   1895,    806,  25557,   2131,
          20005,  34654,   9278,  92683,   2921, 167797,    105]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [108]:
print(sample_output)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-1.0815e-03,  1.1119e-03, -5.9574e-04,  ..., -1.0443e-02,
          -3.2412e-03, -1.6971e-03],
         [ 1.9467e-03,  5.9384e-05, -2.1190e-03,  ..., -1.5625e-02,
           2.3026e-03, -2.0605e-04],
         [ 3.4562e-03,  1.2883e-03,  3.9305e-04,  ..., -1.4371e-02,
          -2.8390e-05,  1.5436e-03],
         ...,
         [-3.5425e-04, -3.2236e-03,  1.2434e-05,  ..., -9.8493e-03,
           1.1274e-03, -2.1251e-03],
         [-1.5258e-03, -2.6940e-04, -1.8443e-03,  ..., -1.9513e-02,
           1.1240e-03, -9.2371e-04],
         [-1.0815e-03,  1.1119e-03, -5.9573e-04,  ..., -1.0443e-02,
          -3.2412e-03, -1.6971e-03]]], grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 9.6244e-03,  1.9249e-02, -7.4001e-03,  1.6524e-02, -8.0338e-03,
         -8.6936e-03, -2.1392e-03, -3.7279e-03, -3.0232e-03, -9.7288e-03,
         -1.0556e-02,  5.9247e-03,  2.4443e-03,  9.2338e-04, -4.9774e-03,
          1.7881e-

In [109]:
print(sample_output.pooler_output.cpu().detach().numpy().shape)

(1, 768)


In [110]:
# Tokenizing Test Input Data

input = []

for i in text_test:
    tokenized_input = tokenizer(
        i,
        padding=True,
        truncation=False,
        return_tensors='pt'
    )
    input.append(tokenized_input)

In [111]:
# Embedding the Test Input Data

output = []

j = 1

for i in range(len(input)):
    
    model_output = model(**input[i])
    model_output = model_output.pooler_output.cpu().detach().numpy()
    output.append(model_output)
    
    # Saving Embedded Test Input Data to Disk

    if ((i + 1) % 200) == 0:
        
        file_name = "./TestData/output" + str(j) + ".pickle"
        
        with open(file_name, "wb") as fp:   
            # Pickling
            pickle.dump(output, fp)
        
        print(file_name + " done")
        output = []
        j += 1
        
file_name = "./TestData/output" + str(j) + ".pickle"

with open(file_name, "wb") as fp:   
    # Pickling
    pickle.dump(output, fp)

print(file_name + " done")
output = []

./TestData/output1.pickle done
./TestData/output2.pickle done
./TestData/output3.pickle done
./TestData/output4.pickle done


In [112]:
# Loading Embedded Input Test Data from Disk

output = []

for i in range(4):
    file_name = "./TestData/output" + str(i + 1) + ".pickle"
    with open(file_name, "rb") as fp:   #Pickling
        file_output = pickle.load(fp)
        for x in file_output:
            output.append(x)
    print(file_name + " done")

X_test = output
output = []

./TestData/output1.pickle done
./TestData/output2.pickle done
./TestData/output3.pickle done
./TestData/output4.pickle done


In [113]:
X_test = np.array(X_test)
X_test.shape

(654, 1, 768)

In [114]:
X_test = X_test.reshape(654, 768)
X_test.shape

(654, 768)

## **Getting Predictions**

In [115]:
# Logistic Regression
lr_test_pred = lr.predict(X_test)

# Support Vector Machine
svm_test_pred = svm.predict(X_test)

# Naive Bayes
nb_test_pred = nb.predict(X_test)

# Stochastic Gradient Descent
sgd_test_pred = sgd.predict(X_test)

# K Nearest Neighbours
knn_test_pred = knn.predict(X_test)

# Decision Tree
dt_test_pred = dt.predict(X_test)

# Random Forest
rf_test_pred = rf.predict(X_test)

In [116]:
# Majority Voting
mv_test_pred = []

for i in range(len(lr_test_pred)):
    one = 0
    zero = 0
    predictions = [nb_test_pred[i], knn_test_pred[i], dt_test_pred[i]]
    for pred in predictions:
        if pred == 1: one += 1
        if pred == 0: zero +=1
    if one > zero: mv_test_pred.append(1)
    else: mv_test_pred.append(0)

mv_test_pred = np.array(mv_test_pred)

## **Evaluating the Predictions**

In [117]:
dataframe_eval = pd.read_csv(
    "Dataset/tamil_offensive_test task 1 with labels.tsv", 
    sep='\t'
)

In [118]:
y_test = dataframe_eval['category']
y_test = LabelEncoder().fit_transform(y_test)

In [119]:
print('Logistic Regression')
print(classification_report(y_test, lr_test_pred))

Logistic Regression
              precision    recall  f1-score   support

           0       0.87      0.96      0.92       536
           1       0.69      0.37      0.48       118

    accuracy                           0.86       654
   macro avg       0.78      0.67      0.70       654
weighted avg       0.84      0.86      0.84       654



In [120]:
print('Support Vector Machine')
print(classification_report(y_test, svm_test_pred))

Support Vector Machine
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       536
           1       0.00      0.00      0.00       118

    accuracy                           0.82       654
   macro avg       0.41      0.50      0.45       654
weighted avg       0.67      0.82      0.74       654



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [121]:
print('Naive Bayes')
print(classification_report(y_test, nb_test_pred))

Naive Bayes
              precision    recall  f1-score   support

           0       0.94      0.74      0.83       536
           1       0.40      0.79      0.53       118

    accuracy                           0.75       654
   macro avg       0.67      0.76      0.68       654
weighted avg       0.84      0.75      0.77       654



In [122]:
print('Stochastic Gradient Descent')
print(classification_report(y_test, sgd_test_pred))

Stochastic Gradient Descent
              precision    recall  f1-score   support

           0       0.86      0.95      0.90       536
           1       0.58      0.30      0.39       118

    accuracy                           0.83       654
   macro avg       0.72      0.62      0.65       654
weighted avg       0.81      0.83      0.81       654



In [123]:
print('K Nearest Neighbours')
print(classification_report(y_test, knn_test_pred))

K Nearest Neighbours
              precision    recall  f1-score   support

           0       0.85      0.97      0.91       536
           1       0.63      0.23      0.34       118

    accuracy                           0.84       654
   macro avg       0.74      0.60      0.62       654
weighted avg       0.81      0.84      0.80       654



In [124]:
print('Decision Tree')
print(classification_report(y_test, dt_test_pred))

Decision Tree
              precision    recall  f1-score   support

           0       0.88      0.84      0.85       536
           1       0.38      0.46      0.42       118

    accuracy                           0.77       654
   macro avg       0.63      0.65      0.64       654
weighted avg       0.79      0.77      0.78       654



In [125]:
print('Random Forest')
print(classification_report(y_test, rf_test_pred))

Random Forest
              precision    recall  f1-score   support

           0       0.83      0.99      0.90       536
           1       0.62      0.11      0.19       118

    accuracy                           0.83       654
   macro avg       0.73      0.55      0.55       654
weighted avg       0.80      0.83      0.77       654



In [126]:
print('Majority Voting')
print(classification_report(y_test, mv_test_pred))

Majority Voting
              precision    recall  f1-score   support

           0       0.89      0.88      0.88       536
           1       0.47      0.48      0.48       118

    accuracy                           0.81       654
   macro avg       0.68      0.68      0.68       654
weighted avg       0.81      0.81      0.81       654

